<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/%EC%9D%BC%EB%B0%98%EB%8C%80%ED%99%94_csv%ED%8C%8C%EC%9D%BC_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

일반 대화를 추가할 때 단순히 추가하는 것이 아니라 기존에 가지고있던 공격대화 데이터의 TF-IDF스코어를 이용해 키워드를 뽑고 추가할 일반 대화 데이터에서 가지고 있는 공격성 키워드와의 일치율을 계산하고 기준점을 넘어가지 않는 대화만 일반대화로 추가

이렇게 하는 이유는 고객센터 대화를 쓴다고 하더라도 공격적인 말을 하는 고객이 있을 수 있기 때문에 걸러주기 위함임.

In [1]:
# 추가할 일반대화들 전부 불러와서 DataFrame으로 저장
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# 카카오톡 대화
import os

dir_path = '/content/drive/MyDrive/AIFFELTHON/일반대화/TS_KAKAO'

file_list = os.listdir(dir_path)

In [5]:
kakao_text_list = []

for i in file_list:
  with open(f'{dir_path}/{i}', 'r') as f:
    list_file = []

    for line in f:
      line = line.strip()
      list_file.append(line)
      
  kakao_text_list.append(list_file)

In [6]:
len(kakao_text_list)

18517

In [11]:
kakao_text_list[1:3]


[['1 : 자격증을 딸까 아니면 지금 하고 있는 일에 집중할까?',
  '2 : 흠 너 무슨 자격증 따려고 하는데?',
  '1 : 나 포토샵 자격증 따려고',
  '2 : 너 전공에서 꼭 필요한 자격증이야?',
  '1 : 음 그런 거 아닌데 있으면 좋을 것 같아서',
  '2 : 나는 전공에 관련된 자격증 위주로 따는 게 좋을 것 같아',
  '1 : 흠 그럼 다른 거 따야겠어',
  '2 : 일도 좋은데 취업 준비를 잘하는 게 우선이지',
  '1 : 그러게... 공부를 해야 하는데 점점 머리가 굳는 느낌이야',
  '2 : 나이 들어서 그래, 아니면 학원은 안 다녀?',
  '1 : 생각보다 학원비가 너무 비싸서 ㅜㅜ 고민',
  '2 : 성인이라 그런지 학원비가 만만치 않아',
  '1 : 응 한 과목에 25만원인데 두 과목 하면 거의 50만원...',
  '2 : 너가 독학할 수 있으면 독학으로 하는 게 좋지',
  '1 : 응응 유튜브 같은 곳 찾아보고 있는 중이야...',
  '2 : 어지간히 중요한 거 아니면 독학으로 해봐!'],
 ['1 : 니는 예능 프로그램 챙겨 보는 거 뭐 있어? 키키',
  '2 : 음 대탈출 챙겨 봤었어요 하하',
  '1 : 나는 놀면 뭐 하니는 잘 챙겨 보는 편이다! 키키',
  '2 : 오호 놀면 뭐 하니 최근에 김태호 피디 그만두지 않았어요?',
  '1 : 오... 대탈출 그게 방탈출 게임 하는 거 맞제? 키키',
  '2 : 넵 하하 방탈출 게임을 모티브로 했죠 하하',
  '1 : 오... 그만둔다고 하던데 벌써 그만둔 건지는 모르겠다 키키',
  '2 : 아하 바뀌면 놀면 뭐 하니 좀 많이 달라질 수도 있겠네요',
  '1 : 대탈출 그거 재밌다고 하더라고?',
  '1 : 시즌이 꽤 많이 나왔다고 하던데?',
  '2 : 시즌4까지 나왔을걸요? 하하',
  '2 : 한 달 전인가 두 달 전에 대탈출 시즌4 끝났어요 하하',
  '1 : 응 그럴 것 같은데...',
  '1 : 김태로랑 유재석 케미가 

In [12]:
# 주제별 대화
# 교육
dir_path_list = []

master_dir_path = '/content/drive/MyDrive/AIFFELTHON/일반대화/TS_EDU'
dir_list = os.listdir(master_dir_path)

for i in dir_list:
  dir_path = f'{master_dir_path}/{i}'
  dir_path_list.append(dir_path)


edu_text_list = []
for i in dir_path_list:
  file_list = os.listdir(i)
  for j in file_list:
    with open(f'{i}/{j}', 'r') as f:
      list_file = []
      for line in f:
        line = line.strip()
        list_file.append(line)
    edu_text_list.append(list_file)

In [13]:
len(edu_text_list)


6393

In [14]:
edu_text_list[1]


['A.무엇을 도와드릴까요 #@소속# 고객센터 상담원 #@이름#입니다',
 'B.상담 좀 하고 싶은데요 어제 #@소속#으로 전학 왔어요',
 'A.고객님 어떤 게 궁금하신가요',
 'B.그러니까 한 텀이라고 해야 되나 그런 건 얼마나 되나요',
 'A.기본적으론 3개월인데요 내신대비 기간 포함 4개월 정도 돼요',
 'B.그럼 3개월 동안 레벨이 올라가고 내려갈 수도 있고 그런가요',
 'A.학생의 레벨하고 맞지 않다면 당연히 조정해야지요',
 'B.그럼 한 반에는 몇 명이나 있나요',
 'A.반마다 다른데요 그래도 일곱에서 열명 내외 그 정도예요',
 'B.그럼 레벨테스트도 있는 거고요',
 'A.그렇습니다 자녀분 테스트 시간에 상담받아보시는 건 어떠세요',
 'B.그럼 아이 일정 조율해서 연락드릴게요 수고하세요',
 'A.문의 사항 있으시면 연락 주세요 #@소속# #@이름# 이었습니다']

In [15]:
# 쇼핑
dir_path_list = []

master_dir_path = '/content/drive/MyDrive/AIFFELTHON/일반대화/TS_Shopping'
dir_list = os.listdir(master_dir_path)

for i in dir_list:
  dir_path = f'{master_dir_path}/{i}'
  dir_path_list.append(dir_path)


shop_text_list = []
for i in dir_path_list:
  file_list = os.listdir(i)
  for j in file_list:
    with open(f'{i}/{j}', 'r') as f:
      list_file = []
      for line in f:
        line = line.strip()
        list_file.append(line)
    shop_text_list.append(list_file)

In [16]:
len(shop_text_list)

13832

In [17]:
shop_text_list[1]

['A.반갑습니다 #@소속# 상담사입니다',
 'B.네 좀 전에 연락 드렸었는데요',
 'B.저는 창고 쪽 담당자이고요 실외기 하얀 연기 때문에요',
 'A.네 고객님 상담 내역 확인 하겠습니다',
 'A.혹시 에어컨 구매 하신 지는 이 년 넘으셨습니까',
 'B.예 이 년 넘었어요',
 'A.네 확인 감사드립니다',
 'A.죄송하지만 현재 상담 지연으로 한 시간 정도 대기 예상됩니다',
 'A.고객님 전화번호 남겨주시면 신속하게 전화 드리겠습니다',
 'B.예 알겠습니다 #@전번#',
 'A.네 고객님 성함 어떻게 되세요',
 'B.예 #@소속# #@이름# 입니다',
 'A.네 알겠습니다 바로 상담 잘 받으시고요',
 'A.즐거운 하루 보내세요',
 'B.네 고맙습니다 수고하세요']

In [18]:
# 민원
dir_path_list = []

master_dir_path = '/content/drive/MyDrive/AIFFELTHON/일반대화/TS_Civil'
dir_list = os.listdir(master_dir_path)

for i in dir_list:
  dir_path = f'{master_dir_path}/{i}'
  dir_path_list.append(dir_path)


civil_text_list = []
for i in dir_path_list:
  file_list = os.listdir(i)
  for j in file_list:
    with open(f'{i}/{j}', 'r') as f:
      list_file = []
      for line in f:
        line = line.strip()
        list_file.append(line)
    civil_text_list.append(list_file)

In [19]:
len(civil_text_list)

9909

In [20]:
civil_text_list[1]

['A.#@소속# 조직도 확인 후 연락드렸습니다.',
 'A.정확한 문의는 #@소속#부서에 문의를 해보시면 좋으실 듯 합니다',
 'B.#@소속#에 조사 부서가 따로 있나요',
 'A.네 맞습니다 따로 있습니다',
 'B.예 그러면 번호를 따로 확인해주시겠습니까',
 'A.네 메모 가능하십니까',
 'B.네 불러 주세요 메모 가능합니다',
 'A.네 #@전번#입니다',
 'B.죄송한데 저희 민원인께서 지금 민원을 먼저 넣으시고요',
 'B.시 담당 공무원에게 민원인의 신분을 일차 피해를 당했다고 합니다',
 'B.가해자한테 민원인 신분을 유출을 당해서 이차 피해를 당하셨거든요',
 'B.그래서 민원인의 가족분이 크게 다치셨고',
 'B.거기에 대해서 사과를 했다고합니다',
 'B.그 부분에 대해서 징계 요청을 하셨다고 합니다',
 'B.하지만 가해자가 높은 사람이라 담당 공무원이 못하겠다 라고 하셨대요',
 'A.위치한 지역에 시청 또는 군청에 감사관에도 문의를 해 본 부분입니까',
 'B.담당 공무원이 가해자가 높은 사람이라서 어렵다고합니다',
 'B.민원인이 일차적으로는 본인의 신분이 유출 되었다고 하는데요',
 'B.그 쪽을 믿을 수가 없대요',
 'B.제가 감사위원회 민원인이 어느 시에서 그렇게 당하셨는지 여쭤봤는데요',
 'B.답변을 못하시겠다라고 하셨어요',
 'A.네 불만이신 경우에는 안내해드린 #@전번#으로 상담해보시면 어떨까요',
 'B.그러면 혹시 연결이 가능할까요',
 'A.네 저희 쪽에서 연결 가능합니다 바로 연결해드리겠습니다',
 'B.예 선생님 혹시 이분 담당자 성함도 알 수 있을까요',
 'A.네 #@이름# 주사입니다',
 'B.네 감사합니다 연결 해주시겠어요',
 'A.네 바로 연결하겠습니다',
 'B.예 감사합니다 수고하세요']

- 일반 대화를 하나의 데이터 프레임으로

In [21]:
# 카카오 대화
kakao_conversation = []

for conv in kakao_text_list:

  conv_list = []
  for sen in conv:
    sen = sen[4:]
    conv_list.append(sen)
  conversation = '\n'.join(conv_list)
  kakao_conversation.append(conversation)

In [23]:
kakao_conversation[1]

'자격증을 딸까 아니면 지금 하고 있는 일에 집중할까?\n흠 너 무슨 자격증 따려고 하는데?\n나 포토샵 자격증 따려고\n너 전공에서 꼭 필요한 자격증이야?\n음 그런 거 아닌데 있으면 좋을 것 같아서\n나는 전공에 관련된 자격증 위주로 따는 게 좋을 것 같아\n흠 그럼 다른 거 따야겠어\n일도 좋은데 취업 준비를 잘하는 게 우선이지\n그러게... 공부를 해야 하는데 점점 머리가 굳는 느낌이야\n나이 들어서 그래, 아니면 학원은 안 다녀?\n생각보다 학원비가 너무 비싸서 ㅜㅜ 고민\n성인이라 그런지 학원비가 만만치 않아\n응 한 과목에 25만원인데 두 과목 하면 거의 50만원...\n너가 독학할 수 있으면 독학으로 하는 게 좋지\n응응 유튜브 같은 곳 찾아보고 있는 중이야...\n어지간히 중요한 거 아니면 독학으로 해봐!'

In [24]:
len(kakao_conversation)

18517

In [25]:
# 교육, 민원, 쇼핑 대화 생성
data_list = [civil_text_list, shop_text_list, edu_text_list]

other_conversation = []

for i in data_list:
  for conv in i:
    conv_list = []
    for sen in conv:
      sen = sen[2:]
      conv_list.append(sen)
    conversation = '\n'.join(conv_list)
    other_conversation.append(conversation)

In [26]:
len(other_conversation)

30134

In [27]:
other_conversation[1]

'#@소속# 조직도 확인 후 연락드렸습니다.\n정확한 문의는 #@소속#부서에 문의를 해보시면 좋으실 듯 합니다\n#@소속#에 조사 부서가 따로 있나요\n네 맞습니다 따로 있습니다\n예 그러면 번호를 따로 확인해주시겠습니까\n네 메모 가능하십니까\n네 불러 주세요 메모 가능합니다\n네 #@전번#입니다\n죄송한데 저희 민원인께서 지금 민원을 먼저 넣으시고요\n시 담당 공무원에게 민원인의 신분을 일차 피해를 당했다고 합니다\n가해자한테 민원인 신분을 유출을 당해서 이차 피해를 당하셨거든요\n그래서 민원인의 가족분이 크게 다치셨고\n거기에 대해서 사과를 했다고합니다\n그 부분에 대해서 징계 요청을 하셨다고 합니다\n하지만 가해자가 높은 사람이라 담당 공무원이 못하겠다 라고 하셨대요\n위치한 지역에 시청 또는 군청에 감사관에도 문의를 해 본 부분입니까\n담당 공무원이 가해자가 높은 사람이라서 어렵다고합니다\n민원인이 일차적으로는 본인의 신분이 유출 되었다고 하는데요\n그 쪽을 믿을 수가 없대요\n제가 감사위원회 민원인이 어느 시에서 그렇게 당하셨는지 여쭤봤는데요\n답변을 못하시겠다라고 하셨어요\n네 불만이신 경우에는 안내해드린 #@전번#으로 상담해보시면 어떨까요\n그러면 혹시 연결이 가능할까요\n네 저희 쪽에서 연결 가능합니다 바로 연결해드리겠습니다\n예 선생님 혹시 이분 담당자 성함도 알 수 있을까요\n네 #@이름# 주사입니다\n네 감사합니다 연결 해주시겠어요\n네 바로 연결하겠습니다\n예 감사합니다 수고하세요'

데이터 프레임 만들기

In [43]:
other_data = pd.DataFrame(other_conversation, columns=['conversation'])

In [44]:
other_data.head()

,conversation
0,반갑습니다 #@소속# #@이름#입니다 무엇을 도와드릴까요\n여보세요 네 안녕하세요 ...
1,#@소속# 조직도 확인 후 연락드렸습니다.\n정확한 문의는 #@소속#부서에 문의를 ...
2,반갑습니다 #@소속# #@이름#입니다 무엇을 도와드릴까요\n제가 주택 임대차로 분...
3,반갑습니다 #@소속# #@이름#입니다 무엇을 도와 드릴까요\n여기요 #@소속#인가...
4,네 #@소속# #@이름#입니다\n안녕하세요 #@소속# #@이름#입니다\n네 안녕하세...


In [56]:
other_data['type'] = '고객센터'

In [57]:
other_data.head()

,conversation,type
0,반갑습니다 #@소속# #@이름#입니다 무엇을 도와드릴까요\n여보세요 네 안녕하세요 ...,고객센터
1,#@소속# 조직도 확인 후 연락드렸습니다.\n정확한 문의는 #@소속#부서에 문의를 ...,고객센터
2,반갑습니다 #@소속# #@이름#입니다 무엇을 도와드릴까요\n제가 주택 임대차로 분...,고객센터
3,반갑습니다 #@소속# #@이름#입니다 무엇을 도와 드릴까요\n여기요 #@소속#인가...,고객센터
4,네 #@소속# #@이름#입니다\n안녕하세요 #@소속# #@이름#입니다\n네 안녕하세...,고객센터


In [58]:
kakao_data = pd.DataFrame(kakao_conversation, columns=['conversation'])

In [59]:
kakao_data.head()

,conversation
0,아니야 싫어\n난 자격증 공부나 할래\n자격증 공부는 좀 돼가...?\n요즘 계속 ...
1,자격증을 딸까 아니면 지금 하고 있는 일에 집중할까?\n흠 너 무슨 자격증 따려고 ...
2,니는 예능 프로그램 챙겨 보는 거 뭐 있어? 키키\n음 대탈출 챙겨 봤었어요 하하\...
3,니는 예능 프로그램 챙겨 보는 거 뭐 있어? 키키\n음 대탈출 챙겨 봤었어요 하하\...
4,자격증을 딸까 아니면 지금 하고 있는 일에 집중할까?\n흠 너 무슨 자격증 따려고 ...


In [60]:
kakao_data['type'] = '카카오'

In [61]:
kakao_data.head()

,conversation,type
0,아니야 싫어\n난 자격증 공부나 할래\n자격증 공부는 좀 돼가...?\n요즘 계속 ...,카카오
1,자격증을 딸까 아니면 지금 하고 있는 일에 집중할까?\n흠 너 무슨 자격증 따려고 ...,카카오
2,니는 예능 프로그램 챙겨 보는 거 뭐 있어? 키키\n음 대탈출 챙겨 봤었어요 하하\...,카카오
3,니는 예능 프로그램 챙겨 보는 거 뭐 있어? 키키\n음 대탈출 챙겨 봤었어요 하하\...,카카오
4,자격증을 딸까 아니면 지금 하고 있는 일에 집중할까?\n흠 너 무슨 자격증 따려고 ...,카카오


In [62]:
data = pd.concat([other_data, kakao_data], ignore_index=True)

In [63]:
len(data)

48651

In [64]:
data.head()

,conversation,type
0,반갑습니다 #@소속# #@이름#입니다 무엇을 도와드릴까요\n여보세요 네 안녕하세요 ...,고객센터
1,#@소속# 조직도 확인 후 연락드렸습니다.\n정확한 문의는 #@소속#부서에 문의를 ...,고객센터
2,반갑습니다 #@소속# #@이름#입니다 무엇을 도와드릴까요\n제가 주택 임대차로 분...,고객센터
3,반갑습니다 #@소속# #@이름#입니다 무엇을 도와 드릴까요\n여기요 #@소속#인가...,고객센터
4,네 #@소속# #@이름#입니다\n안녕하세요 #@소속# #@이름#입니다\n네 안녕하세...,고객센터


In [65]:
data.tail()

,conversation,type
48646,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄 키키\n갑자기? 키키 집이랑 ...,카카오
48647,주말에 보드 타러 갈래? 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거?...,카카오
48648,우리 집 뒤에 중국집 가지마\n왜? 너 거기 좋아했잖아 맛있다며\n다 다음 달까지 ...,카카오
48649,엄마 생신 선물 눈치 게임 실패했다 아 키키\n왜 너희 오빠랑 같이 준비해서 드린다...,카카오
48650,요새 군대 편한가 봄? 옛날이랑 다르네\n왜 그러는데 니 어디 가서 그런 말 하면 ...,카카오


In [66]:
data.duplicated().sum()

656

In [67]:
data.drop_duplicates('conversation', keep='first', inplace=True)

In [68]:
len(data)

47995

In [69]:
data.to_csv('/content/drive/MyDrive/AIFFELTHON/일반대화/normal_conversation.csv')